# Ticket Triage Exploration Notebook

The goal is to understand the data, and do a few experiments testing the use of LLMs for our eventual app

In [ ]:
# Imports 

import pandas as pd # for dataframe handling, CSV reading, etc.
import requests     # for forming HTTP requests
import random       # random number generator

## Set up helper functions 

In [ ]:
""" 
Helper functions
"""

def show_error(err_string: str):
    """
    Print an error message and stop execution
    """
    print(f"Error: {err_string}")
    SystemExit()


def load_data(csv_path: str):
    """
    Load support ticket data from a CSV file.
    
    This function reads a CSV file containing support tickets and returns it as a 
    pandas DataFrame (think of it as a table/spreadsheet in Python).
    
    Returns:
        pd.DataFrame: A table containing all the support tickets with columns like
                     subject, body, priority, language, etc.
        None: If the file can't be found or loaded
    
    Example CSV structure:
        subject        | body            | 
        ---------------|-----------------|
        Login issue    | Can't log in... | 
    """
    # Define the path to our CSV file (relative to where the script is run)
    
    try:
        # Try to read the CSV file into a DataFrame (table)
        df = pd.read_csv(csv_path)
        return df
    except FileNotFoundError:
        # If the file doesn't exist, show an error message to the user
        show_error(f"CSV file not found at {csv_path}")
        return None
    except Exception as e:
        # If any other error occurs, show the error details
        show_error(f"Error loading CSV: {str(e)}")
        return None
    


# Two ways to call the LLM - requests/http vs. SDK

In [ ]:
# Function to call LM Studio LLM
def call_lm_studio_requests(system_content, user_content):
    """
    Send a request to the LM Studio AI server for text analysis/generation.
    
    LM Studio is a local AI server that runs language models on your computer.
    This function sends prompts to it and gets AI-generated responses back.
    
    Args:
        system_content (str): Instructions for how the AI should behave
                             Example: "You are a helpful assistant analyzing support tickets."
        user_content (str): The actual content/question for the AI to process
                           Example: "Translate this text: Bonjour"
    
    Returns:
        str: The AI's response text, or an error message if something goes wrong
    
    How it works:
        1. Prepares a request with the AI model name and messages
        2. Sends the request to LM Studio running on localhost:1234
        3. Extracts and returns the AI's response from the JSON reply
    
    Note: LM Studio must be running locally on port 1234 for this to work
    Change this to whatever LLM provider you decided to use (see slides from class week 1 for ideas)
    """
    # URL where LM Studio's API is listening (localhost = your computer)
    url = "http://localhost:1234/v1/chat/completions"
    
    # Prepare the data to send to the AI
    payload = {
        "model": "openai/gpt-oss-20b",  # The AI model to use
        "messages": [
            # System message: Sets the AI's behavior/role
            {"role": "system", "content": system_content},
            # User message: The actual task/question
            {"role": "user", "content": user_content}
        ],
        "max_tokens": 1200,    # Maximum length of the response
        "temperature": 0.3     # Controls randomness (0=deterministic, 1=creative)
    }
    
    try:
        # Send the request to LM Studio and get the response
        resp = requests.post(url, json=payload, headers={"Content-Type": "application/json"})
        
        # Extract the AI's message from the JSON response
        # The response structure is: {"choices": [{"message": {"content": "AI response here"}}]}
        return resp.json()["choices"][0]["message"]["content"]
    except Exception as e:
        # If anything goes wrong (LM Studio not running, network error, etc.)
        show_error(f"Error: {str(e)}")


# When the SDK is nicer

- Less boilerplate. One call returns parsed objects; streaming is trivial.

- Built-in retries/backoff & connection pooling. Fewer flaky calls on busy GPUs.

- Async support. AsyncOpenAI saves you from wiring aiohttp/httpx yourself.

- Uniform interface across providers. Just swap the base_url to move between LM Studio, OpenAI, Groq, vLLM, etc.

- Newer features surface sooner. (e.g., tool/function calling, JSON schema responses) without hand-crafting payloads.

In [ ]:
# Version 2 of llm provider call using SDK rather than requests:

# remenmber to pip install openai>=1.35 if you want to use the SDK version
from openai import OpenAI
import os
from dotenv import load_dotenv


load_dotenv()  # reads .env in the current working directory to get the groq api key


# Point the SDK at LLM provider

# If we're using LM Studio locally:
#_client = OpenAI(
#    base_url="http://localhost:1234/v1",
#    api_key="lm-studio"  # any non-empty string works for LM Studio
#)

# or use this in to call a model on groq:


_client = OpenAI(
    base_url="https://api.groq.com/openai/v1",
    api_key=os.environ["GROQ_API_KEY"],) # must have a GROQ_API_KEY in the .env file

def call_llm_sdk(      system_content: str,                   
                       user_content: str,                   
                       model: str = "openai/gpt-oss-20b",
                       max_tokens: int = 2000,
                       temperature: float = 0.1,
                       reasoning_effort: str="low",
                       ) -> str:
    """
    Call LLM Provider via the OpenAI SDK (function call) instead of manual HTTP.

    Args:
        system_content: System prompt/instructions.
        user_content: User message.
        model: Model name exposed by provider - defaults to 'openai/gpt-oss-20b', make sure the provider actually has this model or you'll get an error.
        max_tokens: Max tokens to generate - defaults to 1200 tokens; generation will stop when it hits this limit, so be sure to specify sufficent size to allow a response.
        temperature: Sampling temperature - defaults to 0.1 which is meant to be more 'deterministic' and less 'creative'.

    Returns:
        The assistant's reply text. Stops on errors. More sophisticated error handling would be smart!
    """
    try:
        resp = _client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": system_content},
                {"role": "user", "content": user_content},
            ],
            max_tokens=max_tokens,
            temperature=temperature,
            reasoning_effort=reasoning_effort,
        )
        return resp.choices[0].message.content
    except Exception as e:
        # keep your existing helper if you have it; otherwise raise
        try:
            show_error(f"Error calling LLM: {e}")
        except NameError:
            raise


In [ ]:
""" 
Load the csv file into a pandas data frame
"""

csv_file = "IT_Tickets/dataset-tickets-multi-lang_cleaned.csv"
df = load_data(csv_file)


In [ ]:
df

# Now for calling the LLMs

We have our helper functions set up, we've sort of connected to the LLMs, now it's time to put them to use.

Our first attempt will be to simply determine the language of the the Email.  Let's grab a random email from the data frame and display it; then we can use an LLM 
with a good prompt to get the language it is written in.

In [ ]:
# get a random ticket
ticket_number = random.randint(0, len(df))
print(df.iloc[ticket_number])

In [ ]:
# ========== DETECT LANGUAGE ==========
# Prepare prompts for language detection
system_prompt = "You are a helpful assistant analyzing support tickets."

# Create a detailed prompt asking the AI to detect the language
# The triple quotes (###) help the AI understand boundaries
user_prompt = \
f"""Analyze the following support request email and return ONLY the name of the language it is written in, 
return one of the following languages:
German; English; French; Portuguese; Spanish; Unknown;

###
Subject: {df.iloc[ticket_number]['subject']}

Body: {df.iloc[ticket_number]['body']}
###

Your response should be simply one of [German, English, French, Portuguese, Spanish or Unknown], with no additional commentary or charachters; 
If there is more than one language present, choose the predominent one.
"""

# Call the AI to detect language
# ai_response = call_lm_studio_requests(system_prompt, user_prompt)

ai_response = call_llm_sdk(system_prompt, user_prompt, model="openai/gpt-oss-20b", max_tokens=1200, temperature=0.1)

# Clean up the response and extract just the language code

language_code = ai_response.strip().lower() if ai_response else "NO RESPONSE"

print(user_prompt)
print("="*50)
print(f"Detected: {language_code}")


In [ ]:
# use the ticket number from the previous cell so we can work through the ticket fields
# we will use language_code from the prior cell in this prompt to help the LLM do a nice translation.

# ========== Translate the Subject ==========
# Prepare prompts for Subject translation
system_prompt = "You are a helpful assistant whose assigned the job of translating support tickets from the original language to English."

# Create a detailed prompt asking the AI to detect the language
# The triple quotes (###) help the AI understand boundaries
user_prompt = f"""Translate the following email subject line from {language_code} to English, while adapting it to American idioms and phrasing - 
                    your translation should faithfully match the meaning of the original: 

                ###

                {df.iloc[ticket_number]['subject']}

                ###

                your response should be simply be the English translation with no other information."""
                    
 
# Call the AI to detect language
ai_response = call_llm_sdk(system_prompt, user_prompt, model="openai/gpt-oss-20b", reasoning_effort='medium')

# Clean up the response and extract just the language code
subject = ai_response.strip() if ai_response else ""

print(user_prompt)
print("="*50)
print(f"Subject Line: {subject}")

In [ ]:
# use the ticket number from the previous cell so we can work through the ticket fields
# we will use language_code from the prior cell in this prompt to help the LLM do a nice translation.

# ========== Translate the Email Body ==========
# Prepare prompts for Email Body translation
system_prompt = "You are a helpful assistant whose assigned the job of translating support tickets from the original language to English."

# Create a detailed prompt asking the AI to detect the language
# The triple quotes (###) help the AI understand boundaries
user_prompt = \
f"""Translate the following email body from {language_code} to English, while adapting it to American idioms and phrasing - 
your translation should faithfully match the meaning of the original: 

###
{df.iloc[ticket_number]['body']}
###

Your response should be simply be the English translation with no other information.
"""
                    
# Call the AI to detect language
ai_response = call_llm_sdk(system_prompt, user_prompt, model="openai/gpt-oss-20b", reasoning_effort='medium')

# Clean up the response and extract just the language code
body = ai_response.strip() if ai_response else ""

print(user_prompt)
print("="*50)
print(f"Email Body: {body}")

In [ ]:
# HOMEWORK 1
# Add a cell to classify tickets into 4 Types: Incident, Request, Change, Problem - you'll have to craft a prompt that explains to the LLM what thse mean (or maybe not - you can try just asking first)


In [ ]:
# Homework 2
# Add a cell to determine what queue the ticket should be routed to: Billing & Payments, Customer Service, General Inquiry, Human Resources, 
# IT Support, Product Support, Returns & Exchanges, Sales and Pre-Sales, Service Outages, Technical Support
# Remember the concept of one-shot/multi-shot: You could craft a long prompt that gives an example of an email and assignment for each of these categories... 
# Examples are a GREAT way to show an LLM what you want.

In [ ]:
# Homework 3
# Add a cell to determine the priority of the ticket: P1, P2, P3 – read the rules for the priorities and use those rules in your 
# prompt to have the LLM assign the correct priority.
# Again, you can explain the priority levels to the LLM and/or you can provide examples in the prompt. 
# I sketched in the prioritization rules below from the slides


"""Rules for Priority Assignment

P1 - Critical
Security or privacy incident: data breach, ransomware, malware outbreak, phishing success, compromised account, stolen device with sensitive data.
Payment/revenue blocking: checkout/payment API down, failed customer transactions at scale.
Company-wide or regional outage: SSO, VPN, email, network, authentication unavailable.
Safety/legal risk: regulatory or compliance breach, urgent legal exposure.
High impact blocking: department/region/companywide/external customers cannot work, and urgency is blocking or deadline today.

P2 - Major
Medium impact blocking or deadline: a team is blocked, or a single user is blocked with a hard deadline.
Degraded shared service: email delays, slow VPN, partial outage affecting multiple users.
Single user blocking: cannot log in, device will not boot, locked account (no explicit deadline).

P3 - Minor
Informational requests: “how do I…”, “please provide access,” feature requests.
Cosmetic issues, minor bugs, or general inquiries.
Non-blocking tickets with no deadline.

Tie-Breakers
If multiple rules match, select the highest priority (P1 > P2 > P3).
If signals conflict, assign the lower (safer) priority and reduce confidence.

###
Classify the following support request:

Subject: {subject}
Body: {body}
"""


In [ ]:
# Homework 4
# Add a ”final output” that wraps all of the fields (translated subject, translated body, incident type, queue and priority) 
# and readies it to send on to the appropriate queue... 
# If your primary language is not English, try having the LLM translate this final bundle into your language.  
# How was the quality of the translation?  
# If the model isn’t doing a great job, search for a model that’s more tuned for your language (it would also have to know the source languages of course).   


In [ ]:
# Homework 5
# Success criteria:  
# How long does it take YOU to read a ticket and assign these items?  How long does it take the LLM?  
# What’s the speedup?  
# How is the accuracy? 
# Can you find an example where you feel the LLM did a poor job of assigning an Incident type, Queue or Priority?  
# Can you fix it by adjusting your prompt?
